**Building Custom Named Entity Recognition Model Using Spacy**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import spacy

#Installing the required transformer model

In [ ]:
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 51.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-man

#Loading the spacy model as ner_model

In [ ]:
ner_model = spacy.load("en_core_web_trf")
ner_model

In [ ]:
doc = ner_model("Donad Trump was President of USA")

In [ ]:
doc

Donad Trump was President of USA

In [ ]:
type(doc)

spacy.tokens.doc.Doc

In [ ]:
doc.ents

(Donad Trump, USA)

In [ ]:
doc.ents[0], type(doc.ents[0])

(Donad Trump, spacy.tokens.span.Span)

In [ ]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

#Loading custom data
Here we have created data related to finance domain

In [ ]:
import json
with open('/content/drive/MyDrive/PII_tag/financial_data8.json', 'r') as f:
    data = json.load(f)

In [ ]:
data['examples'][100]

{'id': 'b75999c4-0f24-4af6-ace1-319d01dc878b',
 'content': 'Company 0.22% reported 6.31% in Schmidt Group.',
 'metadata': {},
 'annotations': [{'id': 'ce2d5773-c6e8-44e5-bfbc-1b02ab7040b3',
   'start': 8,
   'end': 13,
   'label': 'Percentage',
   'text': '0.22%'},
  {'id': '3d5c841c-9be2-418e-8e37-d29a24fe9ed6',
   'start': 23,
   'end': 28,
   'label': 'Percentage',
   'text': '6.31%'},
  {'id': '32f2d728-152b-46a6-b3a3-7f5d971b8874',
   'start': 32,
   'end': 45,
   'label': 'Organization',
   'text': 'Schmidt Group'}]}

In [ ]:
data['examples'][0].keys()

dict_keys(['id', 'content', 'metadata', 'annotations'])

In [ ]:
data['examples'][0]['content']

'Company 3.47% reported 2023-02-18 in 2022-08-21.'

In [ ]:
data['examples'][0]['annotations']

[{'id': 'd935770c-3460-4555-b547-3f2d2dd147c1',
  'start': 8,
  'end': 13,
  'label': 'Percentage',
  'text': '3.47%'},
 {'id': '25414d11-bb25-4da8-836d-279bd41977cb',
  'start': 23,
  'end': 33,
  'label': 'Date',
  'text': '2023-02-18'},
 {'id': 'a218f443-7800-4859-a95c-d6a433123355',
  'start': 37,
  'end': 47,
  'label': 'Date',
  'text': '2022-08-21'}]

#Preprocessing the data
Creating training data from the custom dataset:
So the training data contains:
*   context
*   start index and end index of the entities
*   Corresponding annotations



In [ ]:
training_data = []
for example in data['examples']:
  temp_dict = {}
  temp_dict['text'] = example['content']
  temp_dict['entities'] = []
  for annotation in example['annotations']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['label'].upper()
    temp_dict['entities'].append((start, end, label))
  training_data.append(temp_dict)

print(training_data[0])

{'text': 'Company 3.47% reported 2023-02-18 in 2022-08-21.', 'entities': [(8, 13, 'PERCENTAGE'), (23, 33, 'DATE'), (37, 47, 'DATE')]}


In [ ]:
print(training_data[2])

{'text': 'New regulations like Palmer-Baker impact financial markets.', 'entities': [(21, 33, 'ORGANIZATION')]}


In [ ]:
training_data[0]['text']

'Company 3.47% reported 2023-02-18 in 2022-08-21.'

In [ ]:
training_data[0]['entities']

[(8, 13, 'PERCENTAGE'), (23, 33, 'DATE'), (37, 47, 'DATE')]

In [ ]:
training_data[0]['text'][29:53]

'2-18 in 2022-08-21.'

#Converting the training data into spacy format
Save it to disk as train.spacy

In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin()

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("/content/drive/MyDrive/PII_tag/train_2k.spacy")

100%|██████████| 2000/2000 [00:00<00:00, 5131.26it/s]


#To make a config file
Config file contains the required hyperparameters
need to be tuned to finetune the model

In [ ]:
# https://spacy.io/usage/training#quickstart

In [ ]:
#!pip install spacy-transformers

#Copy the base configuration file to a config file from which it fetches all the required parameters and hyperparameters

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/PII_tag/base_config.cfg /content/drive/MyDrive/PII_tag/config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/PII_tag/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy debug data /content/drive/MyDrive/PII_tag/config.cfg


============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: en
Training pipeline: transformer, ner
Components from other pipelines: ner, transformer
2000 training docs
2000 evaluation docs
⚠ 1819 training examples also in evaluation data

============================== Vocab & Vectors ==============================
ℹ 23554 total word(s) in the data (1124 unique)
ℹ No word vectors present in the package

========================== Named Entity Recognition ==========================
ℹ 22 label(s)
0 missing value(s) (tokens with '-' label)
⚠ Some model labels are not present in the train data. The model
performance may be degraded for these labels after training: 'PRODUCT',
'PERSON', 'ORDINAL', 'FAC', 'TIME', 'PERCENT', 'ORG', 'EVENT', 'CARDINAL',
'WORK_OF_ART', 'LOC', 'NORP', 'LANGUAGE'.
✔ Good amount of examples for al

# Finetune the model with our custom dataset

In [ ]:
!python -m spacy train /content/drive/MyDrive/PII_tag/config.cfg \
--output /content/drive/MyDrive/PII_tag \
    --paths.train /content/drive/MyDrive/PII_tag/train_2k.spacy \
    --paths.dev //content/drive/MyDrive/PII_tag/train_2k.spacy \
    --gpu-id 0
# --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

In [ ]:
import spacy

# Get the best spacy model which gives result on our train dataset

In [ ]:
nlp_ner = spacy.load("/content/drive/MyDrive/PII_tag/model-best")

In [ ]:
import pickle
from google.colab import files

In [ ]:
output_path = "/content/drive/MyDrive/PII_tag/nlp_ner_model.pkl"
with open(output_path, "wb") as f:
    pickle.dump(nlp_ner, f)

In [ ]:
#doc = nlp_ner(" The Williams Inc increased by $438,533.70 due to 2023-09-19")
#doc=nlp_ner("After securing a lucrative contract with Acme Corporation, the leading tech giant, they reported record profits of $1.5 million in the third quarter of 2023")
#doc_1=nlp_ner("On 2023-05-12, the European Union implemented a new regulation under the GDPR, affecting the trading of 1,000 shares of Bonds, as the Revenue from these investments increased by 8% to $5,000,000.")
doc_2=nlp_ner("On 2023-05-12, the European Union implemented a new regulation under the GDPR. Earnings per Share (EPS)")

colors = {"Organization": "#F67DE3", "Money": "#7DF6D9", "Date":"#a6e22d", "FinancialIndicator": "#8A2BE2","FinancialProduct":"#FFA07A","Percentage":"#FF4500","GPE":"#FF6347","Law":"#DAA520","Quantity":"#4682B4"}
options = {"colors": colors}

spacy.displacy.render(doc_2, style="ent", options= options, jupyter=True)